In [1]:
import pandas as pd, numpy as np, os
import matplotlib.pyplot as plt
import matplotlib.backends.backend_pdf
from matplotlib.backends.backend_pdf import PdfPages

In [2]:
output_dir ='/ihme/costeffectiveness/results/vivarium_ciff_sam/v4.5.6_no_x_factor/ciff_sam/2021_11_29_14_01_14/count_data/'

In [3]:
wasting_pt = pd.read_csv(output_dir +'wasting_state_person_time.csv')
wasting_pt.head()

,Unnamed: 0,sex,year,cause,measure,input_draw,scenario,value,sq_lns,mam_treatment,sam_treatment,age
0,0,female,2022,mild_child_wasting,state_person_time,29,baseline,0.000000,covered,covered,covered,early_neonatal
1,1,female,2022,mild_child_wasting,state_person_time,29,baseline,0.000000,covered,uncovered,covered,early_neonatal
2,2,female,2022,mild_child_wasting,state_person_time,29,baseline,0.000000,covered,covered,uncovered,early_neonatal
3,3,female,2022,mild_child_wasting,state_person_time,29,baseline,0.000000,covered,uncovered,uncovered,early_neonatal
4,4,female,2022,mild_child_wasting,state_person_time,29,baseline,3.096509,uncovered,covered,covered,early_neonatal


In [4]:
wasting_transitions = pd.read_csv(output_dir + 'wasting_transition_count.csv').drop(columns='Unnamed: 0')
wasting_transitions.head()

,sex,year,measure,input_draw,scenario,value,sq_lns,mam_treatment,sam_treatment,age
0,female,2022,mild_child_wasting_to_moderate_acute_malnutrit...,29,baseline,0.0,covered,covered,covered,early_neonatal
1,female,2022,mild_child_wasting_to_moderate_acute_malnutrit...,29,baseline,0.0,covered,uncovered,covered,early_neonatal
2,female,2022,mild_child_wasting_to_moderate_acute_malnutrit...,29,baseline,0.0,covered,covered,uncovered,early_neonatal
3,female,2022,mild_child_wasting_to_moderate_acute_malnutrit...,29,baseline,0.0,covered,uncovered,uncovered,early_neonatal
4,female,2022,mild_child_wasting_to_moderate_acute_malnutrit...,29,baseline,0.0,uncovered,covered,covered,early_neonatal


In [5]:
wasting_transitions.columns

Index(['sex', 'year', 'measure', 'input_draw', 'scenario', 'value', 'sq_lns',
       'mam_treatment', 'sam_treatment', 'age'],
      dtype='object')

# V&V Step 1: wasting remission rate RRs by treatment coverage

Verify the relative risks for MAM and SAM recovery rates by treatment coverage status in the Wasting transition rate relative risks for wasting treatment table. Note that the relative risks will vary between the baseline and alternative scenarios given the reliance of the relative risks on E_MAM and E_SAM values, which vary between simulation scenarios.

## Step 1.1: calculate target RRs

In [6]:
# artifact
from vivarium import Artifact
art = Artifact('/ihme/costeffectiveness/artifacts/vivarium_ciff_sam/ethiopia.hdf',
               filter_terms=['year_start == 2020', f'age_end <= 5'])

In [7]:
[k for k in art.keys if 'cause' not in k and 'population.' not in k and 'birth' not in k]

['metadata.keyspace',
 'metadata.locations',
 'sequela.moderate_acute_malnutrition.disability_weight',
 'sequela.severe_acute_malnutrition.disability_weight',
 'risk_factor.child_wasting.distribution',
 'alternative_risk_factor.child_wasting.distribution',
 'risk_factor.child_wasting.categories',
 'risk_factor.child_wasting.exposure',
 'risk_factor.child_wasting.relative_risk',
 'risk_factor.child_wasting.population_attributable_fraction',
 'risk_factor.child_stunting.distribution',
 'alternative_risk_factor.child_stunting.distribution',
 'risk_factor.child_stunting.categories',
 'risk_factor.child_stunting.exposure',
 'risk_factor.child_stunting.relative_risk',
 'risk_factor.child_stunting.population_attributable_fraction',
 'risk_factor.severe_acute_malnutrition_treatment.exposure',
 'risk_factor.severe_acute_malnutrition_treatment.distribution',
 'risk_factor.severe_acute_malnutrition_treatment.categories',
 'risk_factor.severe_acute_malnutrition_treatment.relative_risk',
 'risk_fac

In [8]:
sam_tx_rr = art.load('risk_factor.severe_acute_malnutrition_treatment.relative_risk').reset_index()
pd.DataFrame(sam_tx_rr.loc[(sam_tx_rr.sex=='Female')&(sam_tx_rr.age_start >1)]
             .set_index([r for r in sam_tx_rr.columns if 'draw' not in r]).mean(axis=1))
# cat1 = untreated
# cat2 = baseline treatment
# cat3 = alternative treatment

0
sex    age_start age_end year_start year_end affected_entity                                    affected_measure parameter          
Female 2.0       5.0     2020       2021     severe_acute_malnutrition_to_mild_child_wasting    transition_rate  cat1       0.000000
                                                                                                                 cat2       1.000000
                                                                                                                 cat3       1.071584
                                             severe_acute_malnutrition_to_moderate_acute_mal... transition_rate  cat1       3.384485
                                                                                                                 cat2       1.000000
                                                                                                                 cat3       0.846121

In [9]:
sam_tx_rr = art.load('risk_factor.moderate_acute_malnutrition_treatment.relative_risk').reset_index()
pd.DataFrame(sam_tx_rr.loc[(sam_tx_rr.sex=='Female')&(sam_tx_rr.age_start >1)]
             .set_index([r for r in sam_tx_rr.columns if 'draw' not in r]).mean(axis=1))
# cat1 = untreated
# cat2 = baseline treatment
# cat3 = alternative treatment

0
sex    age_start age_end year_start year_end affected_entity                                   affected_measure parameter          
Female 2.0       5.0     2020       2021     moderate_acute_malnutrition_to_mild_child_wasting transition_rate  cat1       0.722711
                                                                                                                cat2       1.000000
                                                                                                                cat3       1.007508

In [10]:
sam_tx_rr = art.load('risk_factor.severe_acute_malnutrition_treatment.relative_risk').reset_index()
mam_tx_rr = art.load('risk_factor.moderate_acute_malnutrition_treatment.relative_risk').reset_index()
tx_rrs = pd.concat([sam_tx_rr, mam_tx_rr])
tx_rrs = tx_rrs.loc[(tx_rrs.age_start == 0.5)&(tx_rrs.sex=='Female')].drop(columns=[c for c in tx_rrs.columns if 'draw' not in c and c != 'affected_entity' and c != 'parameter'])
tx_rrs_baseline = tx_rrs.loc[tx_rrs.parameter=='cat1'].drop(columns='parameter').set_index([c for c in tx_rrs.columns if 'draw' not in c and c != 'parameter'])
#tx_rrs_cat2 = tx_rrs.loc[tx_rrs.parameter=='cat2']
tx_rrs_alternative = tx_rrs_baseline / tx_rrs.loc[tx_rrs.parameter=='cat3'].drop(columns='parameter').set_index([c for c in tx_rrs.columns if 'draw' not in c and c != 'parameter'])
tx_rrs_baseline

,draw_0,draw_1,draw_2,draw_3,draw_4,draw_5,draw_6,draw_7,draw_8,draw_9,...,draw_990,draw_991,draw_992,draw_993,draw_994,draw_995,draw_996,draw_997,draw_998,draw_999
affected_entity,,,,,,,,,,,,,,,,,,,,,
severe_acute_malnutrition_to_mild_child_wasting,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
severe_acute_malnutrition_to_moderate_acute_malnutrition,3.327213,4.018068,3.362859,2.961939,2.923948,3.948199,3.022492,3.327790,3.977514,3.292555,...,3.317044,3.236318,2.892277,3.021562,2.847928,3.163777,4.588717,2.982359,2.789703,3.018085
moderate_acute_malnutrition_to_mild_child_wasting,0.725869,0.702035,0.709471,0.716635,0.713935,0.728643,0.745621,0.718438,0.701801,0.709031,...,0.704343,0.758650,0.716917,0.729339,0.721335,0.697261,0.733517,0.742891,0.695907,0.709688


In [11]:
pd.DataFrame(tx_rrs_baseline.mean(axis=1))

,0
affected_entity,
severe_acute_malnutrition_to_mild_child_wasting,0.000000
severe_acute_malnutrition_to_moderate_acute_malnutrition,3.384485
moderate_acute_malnutrition_to_mild_child_wasting,0.722711


In [12]:
tx_rrs_alternative

,draw_0,draw_1,draw_2,draw_3,draw_4,draw_5,draw_6,draw_7,draw_8,draw_9,...,draw_990,draw_991,draw_992,draw_993,draw_994,draw_995,draw_996,draw_997,draw_998,draw_999
affected_entity,,,,,,,,,,,,,,,,,,,,,
severe_acute_malnutrition_to_mild_child_wasting,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
severe_acute_malnutrition_to_moderate_acute_malnutrition,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,...,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000
moderate_acute_malnutrition_to_mild_child_wasting,0.717325,0.717325,0.717325,0.717325,0.717325,0.717325,0.717325,0.717325,0.717325,0.717325,...,0.717325,0.717325,0.717325,0.717325,0.717325,0.717325,0.717325,0.717325,0.717325,0.717325


## Step 1.2: calculate simulation remission RRs

In [13]:
def calculate_wasting_transition_rrs(wasting_transitions, wasting_pt,
                                      ages, transition, source_state, 
                                      groupby_cols):
    transition = (wasting_transitions.loc[(wasting_transitions.age.isin(ages))
                                         & (wasting_transitions.measure==transition)]
                  .groupby(groupby_cols).sum())
    pt = (wasting_pt.loc[(wasting_pt.age.isin(ages))
                       & (wasting_pt.cause==source_state)]
          .groupby(groupby_cols).sum())
    transition_rate = transition / pt
    transition_rate_covered = (transition_rate.reset_index().loc[transition_rate.reset_index()[[c for c in groupby_cols if 'treatment' in c][0]]=='covered']
                               .set_index([c for c in groupby_cols if c != [c for c in groupby_cols if 'treatment' in c][0]])
                               .drop(columns=[c for c in groupby_cols if 'treatment' in c][0]))
    rr = transition_rate / transition_rate_covered
    rr = (rr.groupby([c for c in groupby_cols if c != 'input_draw'])
                       .describe(percentiles=[0.025,0.975]))['value']
    return rr

In [14]:
ages = ['6-11_months', '12_to_23_months', '2_to_4']

# mam to mild (r3)
print('MAM to mild RR target baseline:', rr_mam_to_mild_baseline,
      '\nMAM to mild RR target alternative:', rr_mam_to_mild_alternative)
p = calculate_wasting_transition_rrs(wasting_transitions, wasting_pt, ages, 
                                 'moderate_acute_malnutrition_to_mild_child_wasting_event_count', 
                                 'moderate_acute_malnutrition', 
                                  ['input_draw','scenario','mam_treatment']).reset_index()
p

NameError: name 'rr_mam_to_mild_baseline' is not defined

In [ ]:
# sam to mam (r2)
print('SAM to MAM RR target baseline:', rr_sam_to_mam_baseline,
      '\nSAM to MAM RR target alternative:', rr_sam_to_mam_alternative)
calculate_wasting_transition_rrs(wasting_transitions, wasting_pt, ages, 
                                 'severe_acute_malnutrition_to_moderate_acute_malnutrition_event_count', 
                                 'severe_acute_malnutrition', 
                                  ['input_draw','scenario','sam_treatment'])

In [ ]:
# sam to mild (t1)
print('SAM to mild RR target baseline:', rr_sam_to_mild_baseline,
      '\nSAM to mild RR target alternative:', rr_sam_to_mild_alternative)
calculate_wasting_transition_rrs(wasting_transitions, wasting_pt, ages, 
                                 'severe_acute_malnutrition_to_mild_child_wasting_event_count', 
                                 'severe_acute_malnutrition', 
                                  ['input_draw','scenario','sam_treatment'])

# Step 2: verify that wasting incidence rates do not vary by treatment coverage

(note: this is a validation strategy we had imposed, but should perhaps be rethought?)

In [ ]:
# i3 stratified by mam treatment
calculate_wasting_transition_rrs(wasting_transitions, wasting_pt, ages, 
                                 'susceptible_to_child_wasting_to_mild_child_wasting_event_count', 
                                 'susceptible_to_child_wasting', 
                                  ['input_draw','scenario','mam_treatment'])

In [ ]:
# i3 stratified by sam treatment
calculate_wasting_transition_rrs(wasting_transitions, wasting_pt, ages, 
                                 'susceptible_to_child_wasting_to_mild_child_wasting_event_count', 
                                 'susceptible_to_child_wasting', 
                                  ['input_draw','scenario','sam_treatment'])

In [ ]:
# i2 stratified by mam treatment
calculate_wasting_transition_rrs(wasting_transitions, wasting_pt, ages, 
                                 'mild_child_wasting_to_moderate_acute_malnutrition_event_count', 
                                 'mild_child_wasting', 
                                  ['input_draw','scenario','mam_treatment'])

In [ ]:
# i2 stratified by sam treatment
calculate_wasting_transition_rrs(wasting_transitions, wasting_pt, ages, 
                                 'mild_child_wasting_to_moderate_acute_malnutrition_event_count', 
                                 'mild_child_wasting', 
                                  ['input_draw','scenario','sam_treatment'])

In [ ]:
# i1 stratified by sam treatment
calculate_wasting_transition_rrs(wasting_transitions, wasting_pt, ages, 
                                 'moderate_acute_malnutrition_to_severe_acute_malnutrition_event_count', 
                                 'moderate_acute_malnutrition', 
                                  ['input_draw','scenario','mam_treatment'])

# Step 3: Verifiy remission rates

In [ ]:
# artifact
from vivarium import Artifact
art = Artifact('/ihme/costeffectiveness/artifacts/vivarium_ciff_sam/ethiopia.hdf',
               filter_terms=['year_start == 2020', f'age_end <= 5'])

In [ ]:
def calculate_wasting_transition_rates(wasting_transitions, wasting_pt,
                                      ages, transition, source_state, 
                                      groupby_cols, time_to_response=False):
    transition = (wasting_transitions.loc[(wasting_transitions.age.isin(ages))
                                         & (wasting_transitions.measure==transition)]
                  .groupby(groupby_cols).sum())
    pt = (wasting_pt.loc[(wasting_pt.age.isin(ages))
                       & (wasting_pt.cause==source_state)]
          .groupby(groupby_cols).sum())
    transition_rate = transition / pt
    if time_to_response==True:
        transition_rate = 365 / transition_rate
    transition_rate = (transition_rate.groupby([c for c in groupby_cols if c != 'input_draw'])
                       .describe(percentiles=[0.025,0.975]))['value']
    return transition_rate

In [ ]:
#r4 (30 days)
calculate_wasting_transition_rates(wasting_transitions, wasting_pt, ages, 
                                   'mild_child_wasting_to_susceptible_to_child_wasting_event_count', 
                                   'mild_child_wasting', 
                                      ['input_draw','scenario'], time_to_response=True)

In [ ]:
# r3 
# blend of 63 days untreated and 41 days treated
# looks reasonable
calculate_wasting_transition_rates(wasting_transitions, wasting_pt, ages, 
                                   'moderate_acute_malnutrition_to_mild_child_wasting_event_count', 
                                   'moderate_acute_malnutrition', 
                                      ['input_draw','scenario'], time_to_response=True)

In [ ]:
# r3
# blend of 63 days untreated and 41 days treated
# looks reasonable
calculate_wasting_transition_rates(wasting_transitions, wasting_pt, ages, 
                                   'moderate_acute_malnutrition_to_mild_child_wasting_event_count', 
                                   'moderate_acute_malnutrition', 
                                      ['input_draw','scenario','mam_treatment'], time_to_response=True)

In [ ]:
# r2
# untreated SAM recovery to MAM 
print('Untreated target should be ~> 54\nBased on uncovered rate...\ncovered baseline target: ', 365/((365/57.5) * 0.3),
      '\ncovered alternative target: ',  365/((365/57.5) * 0.275))
calculate_wasting_transition_rates(wasting_transitions, wasting_pt, ages, 
                                   'severe_acute_malnutrition_to_moderate_acute_malnutrition_event_count', 
                                   'severe_acute_malnutrition', 
                                      ['input_draw','scenario','sam_treatment'], time_to_response=True)


In [ ]:
# t1 
# time to recovery effectively treated SAM = 48.3
print('covered baseline target: ', 365 / (365 / 48.3 * 0.7),
      '\ncovered alternative target: ', 365 / (365 / 48.3 * 0.725))

calculate_wasting_transition_rates(wasting_transitions, wasting_pt, ages, 
                                   'severe_acute_malnutrition_to_mild_child_wasting_event_count', 
                                   'severe_acute_malnutrition', 
                                      ['input_draw','scenario','sam_treatment'], time_to_response=True)

# Step 4: validate incidence rates

In [ ]:
#i3 
calculate_wasting_transition_rates(wasting_transitions, wasting_pt, ages, 
                                   'susceptible_to_child_wasting_to_mild_child_wasting_event_count', 
                                   'susceptible_to_child_wasting', 
                                      ['input_draw','scenario'], time_to_response=True)

In [ ]:
#i2
calculate_wasting_transition_rates(wasting_transitions, wasting_pt, ages, 
                                   'mild_child_wasting_to_moderate_acute_malnutrition_event_count', 
                                   'mild_child_wasting', 
                                      ['input_draw','scenario'], time_to_response=True)

In [ ]:
#i2
calculate_wasting_transition_rates(wasting_transitions, wasting_pt, ages, 
                                   'moderate_acute_malnutrition_to_severe_acute_malnutrition_event_count', 
                                   'moderate_acute_malnutrition', 
                                      ['input_draw','scenario'], time_to_response=True)

In [ ]:
# overall transition out of SAM
print('baseline target: 6.7 (95% CI: 5.3-8.4)')
overall_sam = (wasting_transitions.loc[(wasting_transitions.measure.str.contains('severe_acute_malnutrition_to'))
                       & (wasting_transitions.age.isin(ages))]
               .groupby(['input_draw','scenario']).sum()).drop(columns='year')
deaths = pd.read_csv(output_dir + 'deaths.csv')
sam_deaths = deaths.loc[(deaths.wasting_state=='severe_acute_malnutrition')
                       &(deaths.age.isin(ages))].groupby(['input_draw','scenario']).sum()
sam_pt = (wasting_pt.loc[(wasting_pt.cause=='severe_acute_malnutrition')
                        & (wasting_pt.age.isin(ages))]
         .groupby(['input_draw','scenario']).sum())#['value']
overall_sam_remission = (((overall_sam + sam_deaths)/ sam_pt)).groupby('scenario').describe(percentiles=[0.025,0.975])['value']
overall_sam_remission

In [ ]:
365 / 6.7

In [ ]:
365 / 6.87

# look at incidence rates stratified by x-factor exposure

In [ ]:
#i3 
calculate_wasting_transition_rates(wasting_transitions, wasting_pt, ages, 
                                   'susceptible_to_child_wasting_to_mild_child_wasting_event_count', 
                                   'susceptible_to_child_wasting', 
                                      ['input_draw','scenario','x_factor'], time_to_response=False)

In [ ]:
#i2 
calculate_wasting_transition_rates(wasting_transitions, wasting_pt, ages, 
                                   'mild_child_wasting_to_moderate_acute_malnutrition_event_count', 
                                   'mild_child_wasting', 
                                      ['input_draw','scenario','x_factor'], time_to_response=False)

In [ ]:
#i1
calculate_wasting_transition_rates(wasting_transitions, wasting_pt, ages, 
                                   'moderate_acute_malnutrition_to_severe_acute_malnutrition_event_count', 
                                   'moderate_acute_malnutrition', 
                                      ['input_draw','scenario','x_factor'], time_to_response=False)

In [ ]:
#i1
calculate_wasting_transition_rates(wasting_transitions, wasting_pt, ages, 
                                   'moderate_acute_malnutrition_to_severe_acute_malnutrition_event_count', 
                                   'moderate_acute_malnutrition', 
                                      ['input_draw','scenario','x_factor','mam_treatment'], False)